**About** : This notebook is used to infer models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

### Imports

In [ ]:
import os
import gc
import sys
import cv2
import glob
import json
import torch
import shutil
import joblib
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import expit
from joblib import Parallel, delayed

warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from util.logger import Config
from util.metrics import macro_auc
from util.torch import load_model_weights

from data.preparation import prepare_data, prepare_folds
from model_zoo.models import define_model
from inference.predict import infer_onnx, load_sample, infer_sample
from params import CLASSES

### Params

In [ ]:
EVAL = False

In [ ]:
if EVAL:
    DATA_PATH = "../input/train_audio/"
else:
    DATA_PATH = "../input/unlabeled_soundscapes/"

In [ ]:
BATCH_SIZE = 48
USE_FP16 = True
NUM_WORKERS = 4

DEVICE = "cuda"

DURATION = 5
SR = 32000

In [ ]:
FOLD = 0 if EVAL else "fullfit_0"

EXP_FOLDERS = [
    # ("../logs/2024-04-12/8/", [FOLD]),   # LB 0.64 baseline
    # ("../logs/2024-04-18/12/", [FOLD]),  #
    # ("../logs/2024-04-18/15/", [FOLD]),  #
    # ("../logs/2024-04-19/4/", [FOLD]),  # Change norm, sampling
    # ("../logs/2024-04-19/5/", [FOLD]),  # d=15s
    # ("../logs/2024-04-19/7/", [FOLD]),  # minmaxnorm, sampling, nocall, less mix
    # ("../logs/2024-04-19/8/", [FOLD]),  # minmaxnorm, no sampling, nocall, less mix no add
    # ("../logs/2024-04-19/10/", [FOLD]),  # minmaxnorm more mix more aug
    # ("../logs/2024-04-29/2/", [FOLD]),  # minmaxnorm fixed crop
    # ("../logs/2024-04-29/4/", [FOLD]),  # minmaxnorm fixed crop 20s selfmix
    # ("../logs/2024-04-29/6/", [FOLD]),  # minmaxnorm fixed crop no_xc selfmix
    # ("../logs/2024-04-29/7/", [FOLD]),  # minmaxnorm fixed crop no_xc selfmix focal_bce ousmk
    # ("../logs/2024-04-30/0/", [FOLD]),  # minmaxnorm selfmix focal_bce ousmk
    # ("../logs/2024-04-30/1/", [FOLD]),  # minmaxnorm selfmix focal_bce ousmk++
    # ("../logs/2024-04-30/3/", [FOLD]),  # minmaxnorm selfmix focal_bce no_xc more mix
    # ("../logs/2024-04-30/4/", [FOLD]),  # minmaxnorm selfmix bce no_xc ousmk^
    # ("../logs/2024-05-02/0/", [FOLD]),  # selfmix focal_bce ousmk + dedup, new melspec params, start-end sampling
    # ("../logs/2024-05-02/15/", [FOLD]),  # no selfmix focal_bce ousmk + 2nd mask, new melspec params, start-end sampling
    # ("../logs/2024-05-04/4/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 20 epochs, wd AdamW
    # ("../logs/2024-05-04/8/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 30 epochs, wd AdamW, upsample
    # ("../logs/2024-05-04/9/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 30 epochs, wd AdamW, upsample less augs
    # ("../logs/2024-05-05/1/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 20 epochs, wd AdamW new miw
    # ("../logs/2024-05-05/7/", [FOLD], "torch"),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext cls w
    # ("../logs/2024-05-06/1/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix
    # ("../logs/2024-05-06/2/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix b0
    # ("../logs/2024-05-06/3/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix vit-b0
    # ("../logs/2024-05-06/12/", [0]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix vit-b0
    # ("../logs/2024-05-06/18/", [FOLD], "torch"),  # vit-b0 bce ousmk + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix 3ch 
    # ("../logs/2024-05-06/19/", [FOLD], "torch"),  # v2s bce ousmk + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix 3ch 
    # ("../logs/2024-05-06/20/", [FOLD], "torch"),  # v2s bce ousmk + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix 3ch 
    # ("../logs/2024-05-06/22/", [FOLD], "torch"),  # vit-b1
    # ("../logs/2024-05-07/0/", [FOLD], "torch"),  # vit-b0
    # ("../logs/2024-05-07/1/", [f"fullfit_{i}" for i in range(4)], "torch"),  # vit-b0 bs32
    # ("../logs/2024-05-07/4/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 bs64
    # ("../logs/2024-05-07/5/", [f"{i}" for i in range(4)], "torch"),  # vit-b0 repro 0.64
    # ("../logs/2024-05-07/6/", [f"{i}" for i in range(4)], "torch"),  # v2-b0 bs64
    # ("../logs/2024-05-07/7/", [f"{i}" for i in range(4)], "torch"),  # vit-b1 bs64
    # ("../logs/2024-05-10/18/", [f"fullfit_{i}" for i in range(4, 5)], "torch"),  # vit-b0 PL 
    # ("../logs/2024-05-11/10/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL2
    # ("../logs/2024-05-11/11/", [f"fullfit_{i}" for i in range(5)], "torch"),  # effnet-b0 PL2
    # ("../logs/2024-05-12/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL0.72
    # ("../logs/2024-05-12/1/", [f"fullfit_{i}" for i in range(5)], "torch"),  # effnet-b0 PL0.72
    # ("../logs/2024-05-12/2/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b1 PL0.72
    ("../logs/2024-05-13/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PLBirdnet
]

In [ ]:
from util.logger import upload_to_kaggle

upload_to_kaggle(
    [f[0] for f in EXP_FOLDERS],
    directory="../output/dataset_4/",
    dataset_name="BirdCLEF 2024 Weights 4",
    update_folders=True
)

### Preprocessing

In [ ]:
if EVAL:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*/*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

    folds = pd.read_csv('../input/folds_4.csv')
    folds['id'] = folds['filename'].apply(lambda x: x.split('/')[-1][:-4])
    df = df.merge(folds)
    df = df[df['fold'] == 0].reset_index(drop=True)

    df["primary_label"] = df["path"].apply(lambda x:  x.split('/')[-2])
else:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])
    
    # df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))
    # df = df[df["duration"] != 240].reset_index(drop=True)

### Models

In [ ]:
models = []
for e in EXP_FOLDERS:
    try:
        exp_folder, folds, runtime = e
    except:
        exp_folder, folds = e
        runtime = "torch"
    
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    
    for fold in folds:
        weights = exp_folder + f"{config.name}_{fold}.pt"

        model = define_model(
            config.name,
            config.melspec_config,
            head=config.head,
            aug_config=config.aug_config,
            num_classes=config.num_classes,
            n_channels=config.n_channels,
            drop_rate=config.drop_rate,
            drop_path_rate=config.drop_path_rate,
            norm=config.norm if hasattr(config, "norm") else "min_max",
            top_db=config.top_db if hasattr(config, "top_db") else None,
            exportable=config.exportable,
            verbose=True,
            pretrained=False
        )
        model = model.to(DEVICE).eval()

        model = load_model_weights(model, weights, verbose=config.local_rank == 0)
        models.append((model, runtime))

### Export

In [ ]:
sessions = [None for _ in range(len(models))]
if any([runtime != "torch" for _, runtime in models]):
    sessions = []

    import onnx
    import onnxruntime as ort
    from onnxconverter_common import float16

    input_names = ['x']
    output_names = ['output']

    input_tensor = torch.randn(
        1 if EVAL else BATCH_SIZE,
        config.n_channels,
        config.melspec_config['n_mels'],
        313 if config.melspec_config['hop_length'] == 512 else 224
    )

    for i, (model, runtime) in enumerate(models):
        name = f"model_{i}.onnx"
        torch.onnx.export(
            model.encoder.cpu(),
            input_tensor,
            name,
            verbose=False,
            input_names=input_names,
            output_names=output_names,
        )
        onnx_model = onnx.load(name)
        # onnx_model = float16.convert_float_to_float16(onnx_model)
        # onnx.save(onnx_model, f"model_{i}.onnx")
        onnx.checker.check_model(onnx_model)
        ort_session = ort.InferenceSession(f"model_{i}.onnx")

        if runtime == "onnx":
            sessions.append(ort_session)
            print(f'- Convert model {name} to onnx')

        elif runtime == "openvino":
            import openvino.runtime as ov

            !mo --input_model $name # --compress_to_fp16=False
            
            core = ov.Core()
            openvino_model = core.read_model(model='model_0.xml')
            compiled_model = core.compile_model(openvino_model, device_name="CPU")
            infer_request = compiled_model.create_infer_request()
            sessions.append(infer_request)

            print(f'- Convert model {name} to openvino')
        else:
            sessions.append(None)

### Main

In [ ]:
try:
    batches = np.array_split(np.arange(len(df)), len(df) / 1000)
except:
    batches = [np.arange(len(df))]

In [ ]:
# inference_rows = []
# for i, batch in enumerate(batches):
#     print(f"-> Batch {i + 1}/{len(batches)}")
#     df_batch = df.iloc[batch].reset_index(drop=True)

#     waves = joblib.Parallel(n_jobs=os.cpu_count())(
#         joblib.delayed(load_sample)(
#             path, evaluate=EVAL, sr=SR, duration=DURATION, normalize=config.normalize
#         )
#         for path in tqdm(df_batch["path"].values)
#     )
#     all_preds = [
#         infer_sample(
#             wave,
#             models,
#             sessions,
#             device=DEVICE,
#             use_fp16=USE_FP16,
#         )
#         for wave in tqdm(waves)
#     ]

#     del waves
#     gc.collect()

#     for idx in range(len(df_batch)):
#         y_pred = all_preds[idx]
#         preds = expit(y_pred).mean(0)

#         for t, pred in enumerate(preds):
#             predictions = dict([(l, p) for l, p in zip(CLASSES, pred)])
#             inference_rows.append(
#                 {"row_id": f"{df_batch.id[idx]}_{(t + 1) * 5}"} | predictions
#             )

#     del all_preds
#     gc.collect()
#     break

# sub = pd.DataFrame(inference_rows)
# if not EVAL:
#     sub.to_csv(EXP_FOLDERS[0][0] + f'pl_sub_{model_idx}.csv', index=False)
#     print('-> Saved predictions to', EXP_FOLDERS[0][0] + f'pl_sub_{model_idx}.csv')
#     display(sub.head())

### Model per model

In [ ]:
inference_rows = {i : [] for i in range(len(models))}

for i, batch in enumerate(batches):
    print(f"-> Batch {i + 1}/{len(batches)}")
    df_batch = df.iloc[batch].reset_index(drop=True)

    waves = joblib.Parallel(n_jobs=32)(
        joblib.delayed(load_sample)(
            path, evaluate=EVAL, sr=SR, duration=DURATION, normalize=config.normalize
        )
        for path in tqdm(df_batch["path"].values)
    )

    for model_idx in range(len(models)):
        all_preds = [
            infer_sample(
                wave,
                [models[model_idx]],
                sessions,
                device=DEVICE,
                use_fp16=USE_FP16,
            )
            for wave in tqdm(waves)
        ]

        for idx in range(len(df_batch)):
            y_pred = all_preds[idx]
            preds = expit(y_pred).mean(0)

            for t, pred in enumerate(preds):
                predictions = dict([(l, p) for l, p in zip(CLASSES, pred)])
                inference_rows[model_idx].append(
                    {"row_id": f"{df_batch.id[idx]}_{(t + 1) * 5}"} | predictions
                )

        del all_preds
        gc.collect()
    
    del waves
    gc.collect()

    # break

for model_idx in range(len(models)):
    sub = pd.DataFrame(inference_rows[model_idx])
    if not EVAL:
        name = EXP_FOLDERS[0][0] + f'pl_sub_{EXP_FOLDERS[0][1][model_idx]}.csv'
        sub.to_csv(name, index=False)
        print('-> Saved predictions to', name)
        display(sub.head())

    del (sub, inference_rows[model_idx])
    gc.collect()

In [ ]:
# plt.plot([
#     0.6625, 0.7034, 0.8868, 0.8910, 0.9343, 0.9322, 0.9451, 0.9137, 0.9520, 0.9543, 
#     0.9637,0.9348, 0.9656, 0.9688, 0.9695, 0.9702, 0.9713, 0.9709, 0.9709, 0.9709,
# ])
# plt.grid()
# plt.ylim(0.6, 1)
# plt.show()

In [ ]:
if EVAL:
    preds = sub[CLASSES].values
    auc = macro_auc(df["primary_label"].values.tolist()[:len(preds)], preds)
    print(f'Fold 0 AUC: {auc:.5f}')

Done !